# XGBoost Endpoint Versions - Bike Rental Regression
<h4>Invoke Specific Version, Invoke generic endpoint</h4>

In [1]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import os

import boto3
import re
from sagemaker import get_execution_role
import sagemaker

# SDK 2 serializers and deserializers
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

In [2]:
# Create a predictor and point to an existing endpoint (note this endpoint has two versions of the model)
endpoint_name = 'xgboost-bikerental'
predictor = sagemaker.predictor.Predictor (endpoint_name=endpoint_name)
predictor.serializer = CSVSerializer()

In [3]:
# Read Test Data
df_all = pd.read_csv('bike_test.csv')

In [4]:
df_all.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,year,month,day,dayofweek,hour
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027,2011,1,20,3,0
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,1,20,3,1
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,1,20,3,2
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,1,20,3,3
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,1,20,3,4


In [5]:
# Need to pass an array to the prediction
# can pass a numpy array or a list of values [[19,1],[20,1]]
arr_test = df_all[df_all.columns[1:]].values

In [6]:
arr_test.shape

(6493, 13)

### No Variant Specified - Endpoint distributes request based on configured weight

In [7]:
# target_variant = None. No variant preference - endpoint can distribute to all available variants based on weight
result = predictor.predict(arr_test[:5],target_variant=None)
print(result)

b'2.332122325897217,1.9005593061447144,1.6316101551055908,1.0376962423324585,0.9574321508407593'


### Variant Specified - Endpoint routes request to the configured variant

In [8]:
# target_variant = specific variant
result = predictor.predict(arr_test[:5],target_variant='version-0-90-2')
print(result)

b'2.332122325897217,1.9005593061447144,1.6316101551055908,1.0376962423324585,0.9574321508407593'


In [9]:
# target_variant = specific variant
result = predictor.predict(arr_test[:5],target_variant='version-1-2-2')
print(result)

b'2.332122325897217,1.9005593061447144,1.6316101551055908,1.0376962423324585,0.9574321508407593'


### Split the input data into chunks
There are thousands of rows in this data set for which need inference.  
When communicating over internet, it is a good idea to split the data into chunks to prevent payload and timeout error

In [10]:
def inference_by_version(targetVariant = None):
    # For large number of predictions, we can split the input data and
    # Query the prediction service.
    # array_split is convenient to specify how many splits are needed
    predictions = []
    for arr in np.array_split(arr_test,10):
        result = predictor.predict(arr, target_variant=targetVariant)
        result = result.decode("utf-8")
        result = result.split(',')
        print (arr.shape)
        predictions += [float(r) for r in result]
        
    return predictions

### Use all available variants

In [11]:
# None - no preference to variant
targetVariants = [None, 'version-0-90-2', 'version-1-2-2']
countColumns = {}
for variant in targetVariants:
    countColumns[str(variant)] = 'count_' + variant if variant else 'count_all'

In [12]:
for variant in targetVariants:
    print (f'Target Variant: {variant}')    
    df_all[countColumns[str(variant)]] = np.expm1(inference_by_version(variant))

Target Variant: None
(650, 13)
(650, 13)
(650, 13)
(649, 13)
(649, 13)
(649, 13)
(649, 13)
(649, 13)
(649, 13)
(649, 13)
Target Variant: version-0-90-2
(650, 13)
(650, 13)
(650, 13)
(649, 13)
(649, 13)
(649, 13)
(649, 13)
(649, 13)
(649, 13)
(649, 13)
Target Variant: version-1-2-2
(650, 13)
(650, 13)
(650, 13)
(649, 13)
(649, 13)
(649, 13)
(649, 13)
(649, 13)
(649, 13)
(649, 13)


In [13]:
df_all[list(countColumns.values())].describe()

,count_all,count_version-0-90-2,count_version-1-2-2
count,6493.000000,6493.000000,6493.000000
mean,183.412390,183.412390,183.412390
std,171.869936,171.869936,171.869936
min,0.419208,0.419208,0.419208
25%,41.106633,41.106633,41.106633
50%,141.825441,141.825441,141.825441
75%,273.490148,273.490148,273.490148
max,942.930326,942.930326,942.930326


In [14]:
# Delete Endpoint to prevent unnecessary charges
predictor.delete_endpoint()